## Imports

In [1]:
import pandas as pd
from statsmodels.stats import multitest
import matplotlib.pyplot as plt
import pandas as pd
from timers import *

print("Done")

Done


## Prepare dataframe

In [2]:
hive_IDs = {"New Left Hive": '6b5cb8b012cb45038eacc24770a2fff7',
            "Utah OW Nuc Left": '13a71f80590a4184a2956058441c3be3',
            "Utah OW Nuc Right": 'dca02aae55b74ca1b1a6ac35042d2254',
           }

Bee_Weather = BeeWeather()
Bee_Weather.set()

master_df = pd.DataFrame()

for hive_creds in list(hive_IDs.items()):
    current_hive = Hive(*hive_creds)
    current_hive.set()
    if not current_hive.get_weight().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_weight()).rename(columns={"Weight": hive_creds[0] + "_weight"})], axis = 1)
    if not current_hive.get_upper_temp().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_upper_temp()).rename(columns={"Temperature": hive_creds[0] + "_upper_temp"})], axis = 1)
    if not current_hive.get_lower_temp().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_lower_temp()).rename(columns={"Temperature": hive_creds[0] + "_lower_temp"})], axis = 1)
    if not current_hive.get_humidity().empty:
        master_df = pd.concat([master_df, pd.DataFrame(current_hive.get_humidity()).rename(columns={"Humidity": hive_creds[0] + "_humidity"})], axis = 1)

    break
# display(master_df.sort_values(by=['Unix_Time']))
display(Bee_Weather.beeweather)



Processing Broodminder weather data.
Reading Broodminder weather data.
Processing Broodminder data.
Reading Broodminder data.


,Temperature,Precipitation_inches,Relative_Humidity,Cloud_Cover,WindSpeed10mAbove_mph,SurfacePressure_millibars,WindDirection_degrees
Unix_Time,,,,,,,
1721307600,71.7,0.0,44.2,74,4.6,871.7,90.1
1721311200,76.9,0.0,43.0,11,3.2,872.3,164.2
1721314800,82.1,0.0,35.1,0,3.7,872.5,119.3
1721318400,86.8,0.0,28.9,1,2.8,872.3,356.7
1721322000,89.8,0.0,26.5,4,3.5,872.2,132.7
...,...,...,...,...,...,...,...
1721898000,81.5,0.0,42.5,100,7.9,873.3,216.3
1721901600,81.0,0.0,46.0,100,8.0,874.1,262.5
1721905200,79.1,0.0,50.1,98,3.6,874.5,355.2


## Create correlation matrix

In [ ]:
dataframe = pd. DataFrame (data = dataset. data, columns = dataset. feature_names)
dataframe ["target"] = dataset. target

#correlation matrix
matrix = dataframe.corr()
print(matrix)

## Bonferroni Correction of calculated p-values

In [ ]:
# Calculate adjusted p-values
reject, adj_pvals, _, _ = multitest.multipletests(result['p-value'], method='bonferroni')

# Add adjusted p-value to the DataFrame
result['adj.pval'] = adj_pvals

## Display correlation matrix

In [ ]:

dataset = datasets. load_iris()
dataframe = pd. DataFrame(data=dataset. data, columns=dataset. feature_names)
dataframe["target"] = dataset. target
matrix = dataframe.corr()

# plotting correlation matrix
plt.imshow(matrix, cmap='Blues')

# adding colorbar
plt.colorbar()

# extracting variable names
variables = []
for i in matrix.columns:
    variables.append(i)

# Adding labels to the matrix
plt.xticks(range(len(matrix)), variables, rotation=45, ha='right')
plt.yticks(range(len(matrix)), variables)

# Display the plot
plt.show()